In [1]:
%run /home/bb/src/python/valkyrie/lib/valkyrie/apps/common/run_notebook_import.py

/media/veracrypt1/python/valkyrie/lib/valkyrie/apps/common/run_notebook_import.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/media/veracrypt1/python/valkyrie/lib/valkyrie/ml/data.py:291: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [15]:
from valkyrie.quants.linear import calc_auto_corr
from valkyrie.quants.time_average import calc_ema_vol, calc_ems_hl

In [3]:
sdate, edate = '2023-01-01', '2023-01-31'
instrs = ['BDM.BTC.USDT.FP']
freq = '100ms'
ret_n_s = [1, 20, 30 * 20, 10 * 30 * 20]
root_dir = '/home/bb/data/BDM'
#########################################################
#DataMgr
#########################################################
data_mgr = DataMgr(sdate, edate, freq, root_dir, instrs = instrs, ret_n_s = ret_n_s)
df_res = data_mgr.get(instrs[0])

In [4]:
calc_auto_corr(df_res, 'mid_last_ret_1_n', lags = [0, 10, 100, 1000, 10000])

,mid_last_ret_1_n,mid_last_ret_1_n_0,mid_last_ret_1_n_10,mid_last_ret_1_n_100,mid_last_ret_1_n_1000,mid_last_ret_1_n_10000
0.0,1.000000,1.000000,0.007917,0.001290,0.000935,0.000221
10.0,0.007917,0.007917,1.000000,0.002379,-0.000145,-0.001131
100.0,0.001290,0.001290,0.002379,1.000000,-0.000691,-0.000642
1000.0,0.000935,0.000935,-0.000145,-0.000691,1.000000,0.000467
10000.0,0.000221,0.000221,-0.001131,-0.000642,0.000467,1.000000


In [5]:
df_vol = calc_ema_vol(df_res, 'mid_last', freq)

/media/veracrypt1/python/valkyrie/lib/valkyrie/quants/time_average.py:271: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calc_ts_ema" failed type inference due to: No implementation of function Function(<function clip at 0x7fdffc123b00>) found for signature:
 
 >>> clip(float64, Literal[int](0), float64)
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'np_clip': File: numba/np/arrayobj.py: Line 2177.
    With argument(s): '(float64, int64, float64)':
   Rejected as the implementation raised a specific error:
     TypingError: Failed in nopython mode pipeline (step: nopython frontend)
   Unknown attribute 'shape' of type float64
   
   File "../../../../../../home/bb/anaconda3/lib/python3.11/site-packages/numba/np/arrayobj.py", line 2216:
           def np_clip_ss(a, a_min, a_max, out=None):
               <source elided>
               ret = np.empty_like(a) if out is None else o

In [6]:
df_vol.describe()

,var_raw,vol_raw,vol_10,vol_60,vol_300,vol_1800
count,2.677763e+07,2.677763e+07,2.677763e+07,2.677763e+07,2.677763e+07,2.677763e+07
mean,5.190499e-10,6.390511e-02,2.263165e-01,2.487882e-01,2.667326e-01,2.917446e-01
std,2.804653e-08,3.995042e-01,3.353628e-01,3.190476e-01,3.041985e-01,2.801261e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,5.660264e-02,8.919515e-02,1.020929e-01,1.177328e-01
50%,0.000000e+00,0.000000e+00,1.491401e-01,1.748673e-01,1.926715e-01,2.175196e-01
75%,0.000000e+00,0.000000e+00,2.809769e-01,2.964365e-01,3.172679e-01,3.567917e-01
max,6.900545e-05,1.475180e+02,2.907341e+01,1.504040e+01,7.477809e+00,3.741477e+00


In [7]:
df_vol['log_vol_raw'] = df_vol['vol_raw'].apply(np.log)

In [8]:
calc_auto_corr(df_vol, 'log_vol_raw', lags = [0, 10, 100, 1000, 10000])

,log_vol_raw,log_vol_raw_0,log_vol_raw_10,log_vol_raw_100,log_vol_raw_1000,log_vol_raw_10000
0.0,1.000000,1.000000,0.143312,0.107841,0.087544,0.051760
10.0,0.143312,0.143312,1.000000,0.108185,0.085025,0.050132
100.0,0.107841,0.107841,0.108185,1.000000,0.088309,0.051480
1000.0,0.087544,0.087544,0.085025,0.088309,1.000000,0.061847
10000.0,0.051760,0.051760,0.050132,0.051480,0.061847,1.000000


In [9]:
calc_auto_corr(df_vol, 'vol_raw', lags = [0, 10, 100, 1000, 10000])

,vol_raw,vol_raw_0,vol_raw_10,vol_raw_100,vol_raw_1000,vol_raw_10000
0.0,1.000000,1.000000,0.228751,0.187459,0.127051,0.064213
10.0,0.228751,0.228751,1.000000,0.186032,0.126908,0.064124
100.0,0.187459,0.187459,0.186032,1.000000,0.129932,0.064298
1000.0,0.127051,0.127051,0.126908,0.129932,1.000000,0.068946
10000.0,0.064213,0.064213,0.064124,0.064298,0.068946,1.000000


In [10]:
df_vol['vol_raw'] = df_vol['vol_raw'].fillna(0.0)

In [11]:
df_vol.describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99,])

/home/bb/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,var_raw,vol_raw,vol_10,vol_60,vol_300,vol_1800,log_vol_raw
count,2.677763e+07,2.677763e+07,2.677763e+07,2.677763e+07,2.677763e+07,2.677763e+07,2.677763e+07
mean,5.190499e-10,6.390510e-02,2.263165e-01,2.487882e-01,2.667326e-01,2.917446e-01,-inf
std,2.804653e-08,3.995042e-01,3.353628e-01,3.190476e-01,3.041985e-01,2.801261e-01,NaN
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-inf
50%,0.000000e+00,0.000000e+00,1.491401e-01,1.748673e-01,1.926715e-01,2.175196e-01,NaN
75%,0.000000e+00,0.000000e+00,2.809769e-01,2.964365e-01,3.172679e-01,3.567917e-01,NaN
90%,0.000000e+00,0.000000e+00,4.838333e-01,4.983046e-01,5.310457e-01,5.992741e-01,NaN
95%,2.080641e-10,2.561544e-01,6.861991e-01,7.069620e-01,7.451534e-01,8.009933e-01,-1.361975e+00
99%,9.290136e-09,1.711648e+00,1.444523e+00,1.481934e+00,1.495179e+00,1.388351e+00,5.374564e-01
max,6.900545e-05,1.475180e+02,2.907341e+01,1.504040e+01,7.477809e+00,3.741477e+00,4.993950e+00


In [12]:
df_res['net_qty_sum'] = df_res.eval('buy_qty_sum - sell_qty_sum')

In [13]:
df_res.columns

Index(['aq_first', 'aq_last', 'aq_max', 'aq_min', 'apx_first', 'apx_last',
       'apx_max', 'apx_min', 'bpx_first', 'bpx_last', 'bpx_max', 'bpx_min',
       'bq_first', 'bq_last', 'bq_max', 'bq_min', 'buy_px_first',
       'buy_px_last', 'buy_px_max', 'buy_px_min', 'buy_qty_sum',
       'sell_px_first', 'sell_px_last', 'sell_px_max', 'sell_px_min',
       'sell_qty_sum', 'mid_last', 'sprd_last', 'sprd2_last',
       'mid_last_fwd_1_n', 'mid_last_ret_1_n', 'wgt_mid_last_ret_1_n',
       'mid_last_fwd_20_n', 'mid_last_ret_20_n', 'wgt_mid_last_ret_20_n',
       'mid_last_fwd_600_n', 'mid_last_ret_600_n', 'wgt_mid_last_ret_600_n',
       'mid_last_fwd_6000_n', 'mid_last_ret_6000_n', 'wgt_mid_last_ret_6000_n',
       'net_qty_sum'],
      dtype='object')

In [16]:
df_net_qty = calc_ems_hl(df_res, 'net_qty_sum', '100ms')

/media/veracrypt1/python/valkyrie/lib/valkyrie/quants/time_average.py:290: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calc_ts_ems" failed type inference due to: No implementation of function Function(<function clip at 0x7fdffc123b00>) found for signature:
 
 >>> clip(float64, Literal[int](0), float64)
 
There are 2 candidate implementations:
    - Of which 2 did not match due to:
    Overload in function 'np_clip': File: numba/np/arrayobj.py: Line 2177.
      With argument(s): '(float64, int64, float64)':
     Rejected as the implementation raised a specific error:
       TypingError: Failed in nopython mode pipeline (step: nopython frontend)
     Unknown attribute 'shape' of type float64
     
     File "../../../../../../home/bb/anaconda3/lib/python3.11/site-packages/numba/np/arrayobj.py", line 2216:
             def np_clip_ss(a, a_min, a_max, out=None):
                 <source elided>
                 ret = np.empty_like(a)

In [17]:
df = pd.concat([df_net_qty, df_res], axis = 1)

In [27]:
calc_auto_corr(df, ['net_qty_ems_10'], lags = [100, 500, 1000])

ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements

In [19]:
df.columns

Index(['net_qty_ems_10', 'net_qty_ems_60', 'net_qty_ems_300',
       'net_qty_ems_1800', 'aq_first', 'aq_last', 'aq_max', 'aq_min',
       'apx_first', 'apx_last', 'apx_max', 'apx_min', 'bpx_first', 'bpx_last',
       'bpx_max', 'bpx_min', 'bq_first', 'bq_last', 'bq_max', 'bq_min',
       'buy_px_first', 'buy_px_last', 'buy_px_max', 'buy_px_min',
       'buy_qty_sum', 'sell_px_first', 'sell_px_last', 'sell_px_max',
       'sell_px_min', 'sell_qty_sum', 'mid_last', 'sprd_last', 'sprd2_last',
       'mid_last_fwd_1_n', 'mid_last_ret_1_n', 'wgt_mid_last_ret_1_n',
       'mid_last_fwd_20_n', 'mid_last_ret_20_n', 'wgt_mid_last_ret_20_n',
       'mid_last_fwd_600_n', 'mid_last_ret_600_n', 'wgt_mid_last_ret_600_n',
       'mid_last_fwd_6000_n', 'mid_last_ret_6000_n', 'wgt_mid_last_ret_6000_n',
       'net_qty_sum'],
      dtype='object')

In [28]:
import torch
import torch.nn as nn

class LogRatioLayer(nn.Module):
    def __init__(self, channel_idx):        
        super(LogRatioLayer, self).__init__()
        self.channel_idx = channel_idx

    def forward(self, x):                
        # Calculate the log ratio
        log_ratio = torch.log(x[:, self.channel_idx, :, :] / x[:, self.channel_idx, 0, 0])

        return log_ratio

In [32]:
import torch

# Define the LogRatioLayer with a fixed channel index
fixed_channel_idx = 1  # Replace with the desired channel index
log_ratio_layer = LogRatioLayer(channel_idx=fixed_channel_idx)

# Create a sample input tensor
batch_size = 1
num_channels = 2
height = 2
width = 2

# Example input tensor
input_tensor = torch.randn(batch_size, num_channels, height, width)

# Pass the input tensor through the LogRatioLayer
output = log_ratio_layer(input_tensor)

# Print the input and output shapes
print("Input Tensor Shape:", input_tensor.shape)
print("Output Tensor Shape:", output.shape)

# Display the input and output tensors
print("Input Tensor:")
print(input_tensor)
print("\nOutput Tensor:")
print(output)

Input Tensor Shape: torch.Size([1, 2, 2, 2])
Output Tensor Shape: torch.Size([1, 2, 2])
Input Tensor:
tensor([[[[ 7.2314e-01, -4.0741e-01],
          [-7.4451e-02, -5.3526e-02]],

         [[-3.7333e-01,  1.2208e-04],
          [ 3.3920e-01, -2.7453e-01]]]])

Output Tensor:
tensor([[[0., 0.],
         [0., 0.]]])
